In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import glob
import math
import random
import time
import datetime
from collections import defaultdict
from tqdm import tqdm, tqdm_notebook
import xml.etree.ElementTree as ET 
import cv2
import PIL
from keras import backend as K
import gc
import scipy.misc
from utils import *
import tensorflow as tf
from tensorflow.keras import layers
import sys
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
HEIGHT, WIDTH, CHANNEL = 64, 64, 3
BATCH_SIZE = 512
EPOCH = 5000
BUFFER_SIZE = 1024
image_width = HEIGHT
image_height = WIDTH
image_channels = CHANNEL
image_sample_size = 10000
image_output_dir = '../output_images/'
image_input_dir = '../input/all-dogs/all-dogs/'
image_ann_dir = "../input/annotation/Annotation/"

In [ ]:
dog_breed_dict = {}
for annotation in os.listdir(image_ann_dir):
    annotations = annotation.split('-')
    dog_breed_dict[annotations[0]] = annotations[1]

In [ ]:
def get_input_image_dict(image_input_dir, labels_dict):
    image_sample_dict = defaultdict(list)
    for image in os.listdir(image_input_dir):
        filename = image.split('.')
        label_code = filename[0].split('_')[0]
        breed_name = labels_dict[label_code]
        #print('Code: {}, Breed: {}'.format(label_code, breed_name))
        if image is not None:
            image_sample_dict[breed_name].append(image)
    
    print('Created label dictionary for input images.')
    return image_sample_dict
image_sample_dict = get_input_image_dict(image_input_dir, dog_breed_dict)

In [ ]:
len(image_sample_dict)

In [ ]:
def plot_class_distributions(image_sample_dict, title=''):
    class_lengths = []
    labels = []
    total_images = 0
    
    print('Total amount of dog breeds: ', len(image_sample_dict))
    
    for label, _ in image_sample_dict.items():
        total_images += len(image_sample_dict[label])
        class_lengths.append(len(image_sample_dict[label]))
        labels.append(label)
        
    print('Total amount of input images: ', total_images)
        
    plt.figure(figsize = (10,30))
    plt.barh(range(len(class_lengths)), class_lengths)
    plt.yticks(range(len(labels)), labels)
    plt.title(title)
    plt.ylabel('Dog Breed')
    plt.xlabel('Sample size')
    plt.show()
    
    return total_images


#total_images = plot_class_distributions(image_sample_dict)


In [ ]:
def read_image(src):
    img = cv2.imread(src)
    if img is None:
        raise FileNotFoundError
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
def plot_images(directory, image_sample_dict, examples=9, disp_labels=True): 
  
    if not math.sqrt(examples).is_integer():
        print('Please select a valid number of examples.')
        return
    
    imgs = []
    classes = []
    for i in range(examples):
        rnd_class, _ = random.choice(list(image_sample_dict.items()))
        #print(rnd_class)
        rnd_idx = np.random.randint(0, len(image_sample_dict[rnd_class]))
        filename = image_sample_dict[rnd_class][rnd_idx]
        img = read_image(os.path.join(directory, filename))
        imgs.append(img)
        classes.append(rnd_class)
    
    
    fig, axes = plt.subplots(round(math.sqrt(examples)), round(math.sqrt(examples)),figsize=(15,15),
    subplot_kw = {'xticks':[], 'yticks':[]},
    gridspec_kw = dict(hspace=0.3, wspace=0.1))
    
    for i, ax in enumerate(axes.flat):
        if disp_labels == True:
            ax.title.set_text(classes[i])
        ax.imshow(imgs[i])
#plot_images(image_input_dir, image_sample_dict)

In [ ]:
def load_cropped_images(dog_breed_dict=dog_breed_dict, image_ann_dir=image_ann_dir, sample_size=22125, 
                        image_width=image_width, image_height=image_height, image_channels=image_channels):
    curIdx = 0
    breeds = []
    dog_images_np = np.zeros((sample_size,image_width,image_height,image_channels))
    for breed_folder in os.listdir(image_ann_dir):
        for dog_ann in tqdm(os.listdir(image_ann_dir + breed_folder)):
            try:
                img = read_image(os.path.join(image_input_dir, dog_ann + '.jpg'))
            except FileNotFoundError:
                print("nf")
                continue
                
            tree = ET.parse(os.path.join(image_ann_dir + breed_folder, dog_ann))
            root = tree.getroot()
            
            size = root.find('size')
            width = int(size.find('width').text)
            height = int(size.find('height').text)
            objects = root.findall('object')
            for o in objects:
                bndbox = o.find('bndbox') 
                
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)
                
                xmin = max(0, xmin - 4)        # 4 : margin
                xmax = min(width, xmax + 4)
                ymin = max(0, ymin - 4)
                ymax = min(height, ymax + 4)

                w = np.min((xmax - xmin, ymax - ymin))
                w = min(w, width, height)                     # available w

                if w > xmax - xmin:
                    xmin = min(max(0, xmin - int((w - (xmax - xmin))/2)), width - w)
                    xmax = xmin + w
                if w > ymax - ymin:
                    ymin = min(max(0, ymin - int((w - (ymax - ymin))/2)), height - w)
                    ymax = ymin + w
                
                img_cropped = img[ymin:ymin+w, xmin:xmin+w, :]      # [h,w,c]
                # Interpolation method
                if xmax - xmin > image_width:
                    interpolation = cv2.INTER_AREA          # shrink
                else:
                    interpolation = cv2.INTER_CUBIC         # expansion
                    
                img_cropped = cv2.resize(img_cropped, (image_width, image_height), 
                                         interpolation=interpolation)  # resize
                    
                dog_images_np[curIdx,:,:,:] = np.asarray(img_cropped)
                dog_breed_name = dog_breed_dict[dog_ann.split('_')[0]]
                breeds.append(dog_breed_name)
                curIdx += 1
    
    return dog_images_np, breeds

In [ ]:
start_time = time.time()
dog_images_np, breeds = load_cropped_images()
dog_images_np = (dog_images_np)/255
est_time = round(time.time() - start_time)
print("Feature loading time: {}.".format(str(datetime.timedelta(seconds=est_time))))

In [ ]:
def lrelu(x, n, leak=0.2): 
    return tf.maximum(x, leak * x, name=n) 

In [ ]:
def plot_features(features, labels, image_width=image_width, image_height=image_height, 
                image_channels=image_channels,
                examples=25, disp_labels=True): 
  
    if not math.sqrt(examples).is_integer():
        print('Please select a valid number of examples.')
        return
    
    imgs = []
    classes = []
    for i in range(examples):
        rnd_idx = np.random.randint(0, len(labels))
        imgs.append((features[rnd_idx, :, :, :]))
        classes.append(labels[rnd_idx])
    
    
    fig, axes = plt.subplots(round(math.sqrt(examples)), round(math.sqrt(examples)),figsize=(15,15),
    subplot_kw = {'xticks':[], 'yticks':[]},
    gridspec_kw = dict(hspace=0.3, wspace=0.01))
    
    for i, ax in enumerate(axes.flat):
        if disp_labels == True:
            ax.title.set_text(classes[i])
        ax.imshow(imgs[i])

In [ ]:
print('Loaded features shape: ', dog_images_np.shape)
print('Loaded labels: ', len(breeds))

In [ ]:
print('Plotting cropped images by specified coordinates..')
#plot_features(dog_images_np, breeds, examples=4, disp_labels=True)

****PlaceHolder****

In [ ]:
#dog_images_np_tens = tf.placeholder(dtype=tf.float32, shape=[dog_images_np.shape[0],64,64,3])

In [ ]:
w = tf.get_variable('w', [dog_images_np.shape[0],64,64,3])

In [ ]:
w.assign(dog_images_np)


In [ ]:
# images_batch = tf.train.shuffle_batch(
#                                     [dog_images_np], batch_size = BATCH_SIZE,
#                                     num_threads = 4, capacity = 2000 + 3* BATCH_SIZE,
#                                     min_after_dequeue = 200).batch(BATCH_SIZE)
# dog_images_np_tens = tf.data.Dataset.from_tensor_slices(dog_images_np).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dog_images_np_tens = tf.random.shuffle(w,seed = 40)

In [ ]:
dog_images_np_tens = tf.split(dog_images_np_tens,75)#375,59

In [ ]:
dog_images_np_tens

In [ ]:
# del w
# del dog_images_np
import pprint
#pprint.pprint(locals())

In [ ]:
print(dict(zip(locals(),[sys.getsizeof(i)*len(i) for i in locals()])), sep='\n')


In [ ]:

del get_input_image_dict, image_sample_dict, dog_breed_dict, annotations, breeds, dog_images_np, w, plot_class_distributions


In [ ]:
gc.collect()

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def generator( reuse=True):
    input_img = tf.keras.Input(shape = (64,64,3))
    
    x = layers.Conv2DTranspose(256, (3, 3), padding='same', name = "genConvTr1")(input_img)
    x = layers.BatchNormalization(name = "genBn1")(x)
    x = layers.LeakyReLU(alpha = 0.2, name = "genLr1")(x)
    x = layers.Conv2DTranspose(128, (3, 3), padding='same', name = "genConvTr2")(x)
    x = layers.BatchNormalization(name = "genBn2")(x)
    x = layers.LeakyReLU(alpha = 0.2, name = "genLr2")(x)
#     x = layers.Conv2DTranspose(64, (3, 3), padding='same', name = "genConvTr3")(x)
#     x = layers.BatchNormalization(name = "genBn3")(x)
#     x = layers.LeakyReLU(alpha = 0.2, name = "genLr3")(x)
    gen = layers.Conv2DTranspose(3, (3, 3), activation = 'tanh', padding='same', name = "genConvTr_op")(x)
    
    model = tf.keras.Model(input_img, gen)
#     opt = tf.keras.optimizers.RMSprop(lr = 1e-5)#SGD(lr=0.05, momentum=0.5, nesterov=True)
#     #loss_fn = cross_entropy(tf.ones_like(fake_output), fake_output)
#     model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['mae'])
    print(model.summary())
    
    return model

In [ ]:
def discriminator(reuse=True):#input, is_train, reuse=tf.AUTO_REUSE):
    input_img = tf.keras.Input(shape = (64,64,3))
    x = layers.Conv2D(64, (3, 3), padding='same', name = "disConv1")(input_img)
    x = layers.LeakyReLU(alpha = 0.2, name = "disLr1")(x)
    x = layers.Dropout(0.2, name = "disDr1")(x)
    x = layers.Conv2D(128, (3, 3), padding='same', name = "disConv2")(x)
    x = layers.LeakyReLU(alpha = 0.2, name = "disLr2")(x)
    x = layers.Dropout(0.2, name = "disDr2")(x)
    x = layers.Conv2D(128, (3, 3), padding='same', name = "disConv3")(x)
    x = layers.LeakyReLU(alpha = 0.2, name = "disLr3")(x)
    x = layers.Dropout(0.2, name = "disDr3")(x)
    x = layers.Flatten(name = "disFl1")(x)
    dis = layers.Dense(1, name = "disDense1", activation = 'relu')(x)
    
    model = tf.keras.Model(input_img, dis)
#     opt = tf.keras.optimizers.Adam(lr = 1e-4)#SGD(lr=0.05, momentum=0.5, nesterov=True)
#     cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    
#     real_loss = cross_entropy(tf.ones_like(real_output), real_output)
#     fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
#     total_loss = real_loss + fake_loss
#     model.compile(optimizer=opt, loss=total_loss, metrics=['mae'])
    print(model.summary())
    
    return model

In [ ]:
generator_v = generator()

#noise = tf.random.normal([1,64, 64, 3], dtype=tf.dtypes.float32, seed = 40)
#generated_image = generator_v(noise)
#tf.image.convert_image_dtype(generated_image, dtype=tf.dtypes.float32)

discriminator_v = discriminator()
#decision = discriminator_v(generated_image, False)

In [ ]:
random_dim = [64,64,3]

real_image = tf.placeholder(tf.float32, shape = [None, HEIGHT, WIDTH, CHANNEL], name='real_image')
random_input = tf.placeholder(tf.float32, shape=[None, HEIGHT, WIDTH, CHANNEL], name='rand_input')
is_train = tf.placeholder(tf.bool, name='is_train')

fake_image = generator_v(random_input, is_train)

real_result = discriminator_v(real_image, is_train)
fake_result = discriminator_v(fake_image, is_train)

d_loss = tf.reduce_mean(fake_result) - tf.reduce_mean(real_result)  # This optimizes the discriminator.
g_loss = -tf.reduce_mean(fake_result)  # This optimizes the generator.

In [ ]:

t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'dis' in var.name]
g_vars = [var for var in t_vars if 'gen' in var.name]
trainer_d = tf.train.RMSPropOptimizer(learning_rate=2e-4).minimize(d_loss, var_list = d_vars)
trainer_g = tf.train.RMSPropOptimizer(learning_rate=2e-4).minimize(g_loss, var_list = g_vars)
# clip discriminator weights
#d_clip = [v.assign(tf.clip_by_value(v, -0.01, 0.01)) for v in d_vars]

In [ ]:
import sys
print(sys.getsizeof(d_vars))
# del t_vars

In [ ]:
gc.collect()

In [ ]:
from tensorflow.core.protobuf import config_pb2
EPOCH = 50
batch_size = 295
image_batch, samples_num = dog_images_np_tens, 22125

batch_num = int(samples_num / batch_size)
print(batch_num)
total_batch = 0
sess = tf.Session()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer(), options=config_pb2.RunOptions(
        report_tensor_allocations_upon_oom=True))
sess.run(tf.local_variables_initializer(), options=config_pb2.RunOptions(
        report_tensor_allocations_upon_oom=True))
# continue training
version = 0
save_path = saver.save(sess, "/tmp/model.ckpt")
ckpt = tf.train.latest_checkpoint('./model/' + str(version))
saver.restore(sess, save_path)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

print('total training sample num:%d' % samples_num)
print('batch size: %d, batch num per epoch: %d, epoch num: %d' % (batch_size, batch_num, EPOCH))
print('start training...')
for i in range(EPOCH):
    print("Running epoch {}/{}...".format(i, EPOCH))
    for j in tqdm(range(batch_num)):
        gc.collect()
        #print(j)
        d_iters = 5
        g_iters = 1

        train_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 64, 64, 3]).astype(np.float32)
        for k in range(d_iters):
#             print(k)
            train_image = sess.run(image_batch[j])
            #wgan clip weights
#            sess.run(d_clip)

            # Update the discriminator
            _, dLoss = sess.run([trainer_d, d_loss],
                                feed_dict={random_input: train_noise, real_image: train_image, is_train: True})
        #xkcd = gc.collect()
        # Update the generator
        for k in range(g_iters):
            train_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 64, 64, 3]).astype(np.float32)
            _, gLoss = sess.run([trainer_g, g_loss],
                                feed_dict={random_input: train_noise, is_train: True})
        #xkcd = gc.collect()

        # print 'train:[%d/%d],d_loss:%f,g_loss:%f' % (i, j, dLoss, gLoss)

    # save check point every 500 epoch
    if i == 50:
       if not os.path.exists('./model/' + version):
           os.makedirs('./model/' + version)
       saver.save(sess, './model/' +version + '/' + str(i))  
    #if i%50 == 0:
        # save images
     #   if not os.path.exists(newPoke_path):
     #       os.makedirs(newPoke_path)
     #   sample_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
     #   imgtest = sess.run(fake_image, feed_dict={random_input: sample_noise, is_train: False})
        # imgtest = imgtest * 255.0
        # imgtest.astype(np.uint8)
     #   save_images(imgtest, [8,8] ,newPoke_path + '/epoch' + str(i) + '.jpg')

    print('train:[%d],d_loss:%f,g_loss:%f' % (i, dLoss, gLoss))
coord.request_stop()
coord.join(threads)

In [ ]:
def test():
    random_dim = 100
    with tf.variable_scope('input'):
        real_image = tf.placeholder(tf.float32, shape = [None, HEIGHT, WIDTH, CHANNEL], name='real_image')
        random_input = tf.placeholder(tf.float32, shape=[None, 64, 64, 3], name='rand_input')
        is_train = tf.placeholder(tf.bool, name='is_train')
    
    # wgan
    fake_image = generator(random_input, random_dim, is_train)
    real_result = discriminator(real_image, is_train)
    fake_result = discriminator(fake_image, is_train, reuse=True)
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer(), options=config_pb2.RunOptions(
        report_tensor_allocations_upon_oom=True)
    variables_to_restore = slim.get_variables_to_restore(include=['gen'])
    print(variables_to_restore)
    saver = tf.train.Saver(variables_to_restore)
    ckpt = tf.train.latest_checkpoint('./model/' + version)
    saver.restore(sess, ckpt)

In [ ]:
test()

In [ ]:
import zipfile 
from PIL import Image 

In [ ]:
def getDog(seed, index):
        
#         xx = np.zeros((64,64,3))
#         xx[index] = 0.70
#         xx[np.random.uniform(-1.0, 1.0, size=[batch_size, 64, 64, 3]).astype(np.float32)] = 0.30
#         train_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 64, 64, 3]).astype(np.float32)
        img = generator_v.predict(seed, verbose = 1)
        img = img*255
        index = (index+1)%10000
        return Image.fromarray( img.astype('uint8') ) 

In [ ]:
z = zipfile.PyZipFile('imagesx.zip', mode='w')
d = DogGenerator()
for k in range(10):
    img = d.getDog(np.random.uniform(-1.0, 1.0, size=[batch_size, 64, 64, 3]).astype(np.float32), k)
    f = str(k)+'.png'
    img.save(f,'PNG'); z.write(f); os.remove(f)
    #if k % 1000==0: print(k)
z.close()

In [ ]:
!unzip imagesx.zip -d /opt
for _ in list(os.listdir('/opt')[2:5]):
    img = plt.imread('/opt/'+_)
    plt.imshow(img)
    